In [1]:
#数值运算包
import math
import numpy as np

#文件读写包
import pandas as pd
import os
import csv

#导入进度条功能包
from tqdm import tqdm

#导入pytorch包及其所需的模块
import torch
import torch.nn as nn 
from torch.utils.data import Dataset,DataLoader, random_split

#导入学习曲线绘制模块
from torch.utils.tensorboard import SummaryWriter

In [2]:
#same_seed() 函数为神经网络的训练提供一致的随机种子，确保训练结果的可复现性。它的输入 seed 是一个整数，我们可以在 2.7. 中的config里设置它。
#train_valid_split() 函数可以根据我们给定的验证集比例（valid_ratio）将原始的训练集随机划分为训练集和验证集，以供训练过程使用。它需要 3 个输入参数：
# 未分割的训练集（data_set），验证集比例（valid_ratio）和随机种子数（seed）。加入人工设置的随机种子数的目的也是为了使得分割方式在每一次训练的尝试中保持一致，使模型的训练结果有更强的可比性。
#predict() 函数即是模型测试所用函数。我们需要对其输入测试集（test_loader），训练好的模型（model）和跑模型的设备（device）。其输出值为我们训练好的模型对测试集的预测结果。

In [3]:
#定义一些工具：
def same_seed(seed):
    # 使用确定性算法(deterministic algorithms)，以确保相同的input，parameters和环境可以输出相同的output，使得训练结果可以复现。
    torch.backends.cudnn.deterministics = True
    # 由于使用GPU进行训练时，cuDNN会自动选择最高效的算法，导致训练结果难以复现，因此需要关闭benchmark模式。
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) # 根据输入的seed设置固定的numpy seed。
    if torch.cuda.is_available():  # 根据输入的seed值在torch中设置固定的起始参数。
        torch.cuda.manual_seed_all(seed)

def train_valid_spilt(data_set, vaild_ratio, seed):
    vaild_set_size = int(vaild_ratio * len(data_set))
    train_set_size = len(data_set) - vaild_set_size
    train_set,vaild_set = random_split(data_set,
                                        [vaild_set_size,train_set_size],
                                        generator=torch.Generator().manual_seed(seed))
    return np.array(train_set),np.array(vaild_set)

def predict(test_loader, model, device):
    model.eval()
    preds = []
    for x in tqdm(test_loader): # tqmd可作为for循环迭代器使用，同时也提供进度条服务。
        x = x.to(device)
        with torch.no_grad():
            pred = model(x)
            preds.append(pred.detach().cpu())# detach()函数从原tensor中剥离出一个新的相等tensor，并将新tensor放入cpu。
    preds = torch.cat(preds,dim=0).numpy() # 将preds列表拼接成tensor，再转化为np array。
    return preds


In [4]:
#编写Dataset类
class COVID19Dataset(Dataset):
    def __init__(self,x,y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x =  torch.FloatTensor(x)

    def __getitem__(self,idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx],self.y[idx]
    
    def __len__(self):
        return len(self.x)

In [5]:
class MyModel(nn.Module):
    def __init__(self,input_dim):
        super(MyModel,self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(Linear(input_dim, 64)),
            nn.ReLU(),
            nn.Linear(Linear(64, 32)),
            nn.ReLU(),
            nn.Linear(Linear(32, 16)),
            nn.ReLU(),
            nn.Linear(Linear(16, 8)),
            nn.ReLU(),
            nn.Linear(Linear(8, 1)),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x